In [1]:
import gym
import tensorflow as tf
import numpy as np
import pprint



In [15]:
player_r = Player("R")
player_j = Player("J")


model=tf.keras.models.load_model("my_model", compile=False)
jeux = create_board(6, 7)
done = verification_plateau(jeux) 
g = 2 



while not done:
    if g % 2 == 0:
        valeurs_q=model(np.expand_dims(jeux, axis=0))
        hit=int(tf.argmax(valeurs_q[0], axis=-1))
        print(hit)
        if hit not in player_r.free_cols(jeux):
            print('Erreur de coup')
            break
        else:
            jeux = player_r.drop_piece(jeux, hit)
    else : 
        pprint.pprint(reveal_plateau(jeux))
        hit = int(input('Quelle colonne ?'))
        jeux = player_j.drop_piece(jeux, hit)
    
    g+=1
    done = verification_plateau(jeux)
            
if g % 2 == 0:
    print("Les jaunes ont gagnés")
else:
    print("Les rouges ont gagnés")

3
[['.', '.', '.', 'R', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.']]
Quelle colonne ?2
3
[['.', '.', 'J', 'R', '.', '.', '.'],
 ['.', '.', '.', 'R', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.']]
Quelle colonne ?3
1
[['.', 'R', 'J', 'R', '.', '.', '.'],
 ['.', '.', '.', 'R', '.', '.', '.'],
 ['.', '.', '.', 'J', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.']]
Quelle colonne ?2
1
[['.', 'R', 'J', 'R', '.', '.', '.'],
 ['.', 'R', 'J', 'R', '.', '.', '.'],
 ['.', '.', '.', 'J', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.']]
Quelle colonne ?2
4
[['.',

In [3]:
def create_board(nb_row, nb_col):
    if nb_row <= 0 or nb_col <= 0:
        raise ValueError()
    matrix = [['.'for j in range(nb_col)] for i in  range(nb_row)]
    return np.array(matrix)


def free_cols(plateau):
    index_nb = []
    i = 0
    while i < len(plateau[-1]):
        if plateau[-1][i] == '.':
            index_nb.append(i)
        i += 1
    return index_nb

def drop_piece(plateau, col, couleur):
    if couleur not in "RJ":
        raise ValueError("la couleur n'existe pas")
    if col not in free_cols(plateau):
        raise ValueError("La colonne est déjà remplie")
    if col < 0:
        raise ValueError("L’index de la colonne est < 0")
    i = 0
    while i < len(plateau):
        if plateau[i][col] == '.':
            plateau[i][col] = couleur
            break
        i += 1
    return plateau

def row_win(plateau, couleur):
    if couleur not in "RJ":
        raise ValueError("La couleur n’existe pas")
    i=0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0; c = j
            while c < len(plateau[i]) and plateau[i][c] == couleur:
                a += 1
                c += 1
                if a == 4:
                    return True     
            j += 1
        i += 1
    return False

def col_win(plateau, couleur):
    if couleur not in "RJ":
        raise ValueError("La couleur n’existe pas")
    i = 0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0
            h = i
            while h < len(plateau) and plateau[h][j] == couleur:
                a += 1
                h += 1
                if a == 4:
                    return True
            j += 1
        i += 1
    return False

def diag_win(plateau, couleur):
    if couleur not in "RJ":
        raise ValueError("La couleur n’existe pas")
    i = 0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0; b = 0
            c = j; c1 = j
            r = i; r1 = i
            while r < len(plateau) and c < len(plateau[r]) and plateau[r][c] == couleur:
                r += 1
                c += 1
                a += 1
                if a == 4:
                    return True
            while r1 < len(plateau) and c1 >= 0 and plateau[r1][c1] == couleur:
                r1 += 1
                c1 -= 1
                b += 1
                if b == 4:
                     return True
            j += 1
        i += 1
    return False

In [4]:
def play_random(plateau, couleur):
    for i in plateau:
        if len(i) != 7 and len(plateau) != 6:
            raise ValueError("Erreur plateau")
    if len(free_cols(plateau)) == 0:
        raise ValueError("Plateau rempli")
    if couleur not in "RJ":
        raise ValueError("la couleur n'existe pas")
    col = random.choice(free_cols(plateau))
    return drop_piece(plateau, col, couleur)

In [5]:
class Player:

    def __init__(self, color):
        self.color = color
    
    def free_cols(self, plateau):
        index_nb = []
        i = 0
        while i < len(plateau[-1]):
            if plateau[-1][i] == 0:
                index_nb.append(i)
            i += 1
        return index_nb

    def drop_piece(self, plateau, col):
        if self.color not in "RJ":
            raise ValueError("la couleur n'existe pas")
        if col not in self.free_cols(plateau):
            raise ValueError("La colonne est déjà remplie")
        if col < 0:
            raise ValueError("L’index de la colonne est < 0")
        i = 0
        while i < len(plateau):
            if plateau[i][col] == 0:
                if self.color == 'R':
                    plateau[i][col] = 1
                    break
                else:
                    plateau[i][col] = -1
                    break
                    
            i += 1
        return plateau
    
    def play_random(self, plateau):
        for i in plateau:
            if len(i) != 7 and len(plateau) != 6:
                raise ValueError("Erreur plateau")
        if len(self.free_cols(plateau)) == 0:
            raise ValueError("Plateau rempli")
        if self.color not in "RJ":
            raise ValueError("la couleur n'existe pas")
        col = random.choice(self.free_cols(plateau))
        return self.drop_piece(plateau, col), col
    
    def my_color(self):
        return self.color

In [6]:
def row_win(plateau, couleur):
    i=0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0; c = j
            while c < len(plateau[i]) and plateau[i][c] == couleur:
                a += 1
                c += 1
                if a == 4:
                    return True     
            j += 1
        i += 1
    return False

def col_win(plateau, couleur):
    i = 0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0
            h = i
            while h < len(plateau) and plateau[h][j] == couleur:
                a += 1
                h += 1
                if a == 4:
                    return True
            j += 1
        i += 1
    return False

def diag_win(plateau, couleur):
    i = 0
    while i < len(plateau):
        j = 0
        while j < len(plateau[i]):
            a = 0; b = 0
            c = j; c1 = j
            r = i; r1 = i
            while r < len(plateau) and c < len(plateau[r]) and plateau[r][c] == couleur:
                r += 1
                c += 1
                a += 1
                if a == 4:
                    return True
            while r1 < len(plateau) and c1 >= 0 and plateau[r1][c1] == couleur:
                r1 += 1
                c1 -= 1
                b += 1
                if b == 4:
                     return True
            j += 1
        i += 1
    return False


def verification_plateau(matrix):
    if row_win(matrix,1) == False and row_win(matrix,-1) == False and diag_win(matrix, -1) == False \
        and diag_win(matrix, 1) == False and col_win(matrix, 1) == False and col_win(matrix, -1) == False:
        
        return False
    else : 
        return True 
    
    
def create_board(nb_row, nb_col):
    if nb_row <= 0 or nb_col <= 0:
        raise ValueError()
    matrix = [[0 for j in range(nb_col)] for i in  range(nb_row)]
    return np.array(matrix)
    
    

In [7]:
def reveal_plateau(test):
    matrixx = [['.'for j in range(test.shape[1])] for i in  range(test.shape[0])]
    for i in range(test.shape[0]):
        for j in range(test.shape[1]):
            if test[i][j] == 1:
                matrixx[i][j] = "R"
            elif test[i][j] == -1:
                matrixx[i][j] = "J"
                
    return matrixx